In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

import os
from langchain_openai import AzureChatOpenAI


os.environ["AZURE_OPENAI_API_KEY"] = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2024-12-01-preview"

llm = AzureChatOpenAI(
    azure_deployment= "gpt-5-mini",
    api_version = "2024-12-01-preview",
    azure_endpoint = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/",
    api_key = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9",
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

#Entry point
graph_builder.add_edge(START, "chatbot")

#Exit point
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

#Visualizer
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [3]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: I’m sorry—you sound really upset. That’s a valid feeling. Do you want to tell me what happened?

If you want to calm down right now, here are a few quick things you can try (pick one that feels doable):

- Slow breathing: inhale for 4 seconds, hold 4, exhale for 6. Repeat a few times.  
- Grounding (5-4-3-2-1): name 5 things you see, 4 things you can touch, 3 things you hear, 2 things you smell, 1 thing you taste.  
- Move: a brisk walk, jumping jacks, or stomping for a minute to shift physical energy.  
- Name it: say to yourself “I’m feeling angry” and rate it 0–10. Naming often reduces intensity.  
- Short timeout: step away, set a timer for 10–20 minutes, and come back when you feel calmer.  
- Write one sentence: what made you angry and what you want to happen instead.

If you’d like help with next steps, I can:
- Listen if you want to vent,
- Help you plan what to say (I-statements, clear requests),
- Guide you through a breathing or grounding exercise now,
- Suggest l